In [788]:
import os
import json
import numpy as np
import timeit
from copy import deepcopy

from scipy.sparse import csr_matrix

In [793]:
class MatrixUtil:
    def __init__(self):
        with open("../datasets/ecom_ratings/customers.json") as f_:
            self.users = json.load(f_)

        with open("../datasets/ecom_ratings/products.json") as f_:
            self.products = json.load(f_)

        with open("../datasets/ecom_ratings/ratings.json") as f_:
            self.ratings = json.load(f_)
            
        self.product_id_to_idx = {
            product['Id']: idx
            for idx, product in enumerate(self.products)
        }
        
        self.idx_to_product_id = {
            idx: id_ for id_, idx in self.product_id_to_idx.items()
        }

        self.user_id_to_idx = {
            user['Id']: idx
            for idx, user in enumerate(self.users)
        }
        
        self.idx_to_user_id = {
            idx: id_ for id_, idx in self.user_id_to_idx.items()
        }
        
        self.matrix = self._generate_matrix()
        
    def _generate_matrix(self):
        user_data = []
        product_data = []
        rating_data = []

        for rating in ratings:
            rating_data.append(rating['Rate'])    
            user_data.append(self.user_id_to_idx[rating['CustomerID']])
            product_data.append(self.product_id_to_idx[rating['ProductID']])

        matrix = csr_matrix((rating_data, (user_data, product_data)), 
                            shape=(len(self.users), len(self.products))).toarray()
        return matrix
    
    def get_user_vector(self, id_):
        user_idx = self.user_id_to_idx.get(id_)
        if user_idx == None:
            raise ValueError("Provided id exist does not exist")
        return self.matrix[user_idx][:]
    
    def get_item_vector(self, id_):
        item_idx = self.product_id_to_idx.get(id_)
        if item_idx:
            raise ValueError("Provided id exist does not exist")
        
        return self.matrix[item_idx][:]

In [794]:
mutil = MatrixUtil()

130754


MemoryError: Unable to allocate 127. GiB for an array with shape (130754, 130754) and data type int64

In [60]:
vector = mutil.get_user_vector(103954)

In [13]:
# The end goal is often either to:
# 1. Predict a user's rating value of an item
# 2. Predict the top-k items

# Important to note that the first goal can be used to produce the second goal, albeit being less efficient.

# User-based collaborative filtering

The ratings provided by similar users to a target user are used to make recommendations for that user. The weighted average values of those similar users on an item k is used as the predicted rating for the target user on that item k.

### Issues

1. Users may have different scales as one user may be biased towards liking most items and another user may be biased towards not liking at all.
2. Users may have liked different items.

To fix issue two, you take an intersection of the set of items liked by both users and use this to calculate similarity.

### Finding similar users using Pearson correlation coefficient.

The similarity metric uses items that have been rated by the target users and potential similar users as a vector.

(1)
$$ 
µ_{u} = \frac{\sum_{k∈I_u} r_{uk}}{|I_u|}
$$


(1.1)
$$ 
Sim(u, v) = Pearson(u,v) = \frac{\sum_{k∈I_{u}∩I_{v}}(r_{uk} - µ_u) * (r_{vk} - µ_v)}{\sqrt{\sum_{k∈I_{u}∩I_{v}}(r_{uk} - µ_u)^2} * \sqrt{\sum_{k∈I_{u}∩I_{v}}(r_{vk} - µ_v)^2}}
$$


(1.2)
$$ 
Sim(u, v) = Cosine(u,v) = \frac{\sum_{k∈I_{u}∩I_{v}}r_{uk} * r_{vk}}{\sqrt{\sum_{k∈I_{u}∩I_{v}}{r_{uk}}^2} * \sqrt{\sum_{k∈I_{u}∩I_{v}}{r_{vk}}^2}}
$$


Where:

$I_u$ is the set of items rated by user u.

$r_{uk}$ is the rating a user u gives to an item k.

$µ_{u}$ is the mean of the ratings for user u.

The replacement of u with v in the variables above translates to the user v.

### Cosine and Pearson Correlation

Cosine similarity checks for the angular difference between two vectors in relation to the origin. This means that vectors [1, 1, 1, 1] (let's call it A) and [500, 500, 500, 500] (let's call it B) will have an angle of 0 despite having different magnitudes. Pearson correlation checks for a linear relationship between two datasets (in this case vectors A and vectors B are considered different distributions rather than a single). For example, if an increase in a variable in dataset a also leads to an increase in its corresponding item in dataset b, that can be considered to be a positive correlation. This concept of correlation can then be transferred to check the similarity between two vectors. Since Pearson correlation can be used to check for the relationship between two vectors, it is more discriminative compared to cosine similarity.

References: https://blogs.sas.com/content/iml/2019/09/03/cosine-similarity.html, https://stats.stackexchange.com/questions/235673/is-there-any-relationship-among-cosine-similarity-pearson-correlation-and-z-sc, https://www.geeksforgeeks.org/python-pearson-correlation-test-between-two-variables/, https://leimao.github.io/blog/Cosine-Similarity-VS-Pearson-Correlation-Coefficient/


### Question on calculating the mean of ratings...

Since we are using the intersection of items rated between users. Should we not use then intersection as well when calculating the mean of a ratings for user u instead of using all of the items rated?

#### Answer...

1. It can be computationally expensive to calculate mean for each user u and v combination.
2. It is hard to argue that the approach of using the intersection being better than using all the items or vice-versa.
3. In cases where the intersection between two users is only 1 item, the similarity metric will fail because the part of eqn (1.1) that says $ (r_{uk} - µ_u) $ as that will yield zero.

In [789]:
def average_rating(vector):
    ratings = vector[vector != 0]
    return round(ratings.sum() /len(ratings), 6)

In [ ]:
# Note:
    
# How Numpy einsum works:
# https://ajcr.net/Basic-guide-to-einsum/

In [768]:
def cos_sim_score(u_vec, v_vec):   
    u_ratings_idx = np.where(u_vec != 0)
    v_ratings_idx = np.where(v_vec != 0)
    
    intersecting_idx = np.intersect1d(u_ratings_idx, v_ratings_idx)
    u_ratings = u_vec[intersecting_idx]
    v_ratings = v_vec[intersecting_idx]
        
    num = np.multiply(u_ratings, v_ratings).sum()
    
    if num == 0:
        return 0

    u_mag = np.sqrt(np.dot(u_ratings, u_ratings))
    v_mag = np.sqrt(np.dot(v_ratings, v_ratings))
    
    denum = u_mag * v_mag
    
    return round(num / denum, 6)

def cos_sim_scores(u_vec, matrix):
    matrix = deepcopy(matrix)
    u_matrix = np.array([u_vec] * matrix.shape[0])
    
    for idx, val in enumerate(u_vec):
        if val == 0:
            matrix[:, idx] = 0      
    u_matrix[matrix[:, :] == 0] = 0
    
    # Using einsum to calculate the diagonals of matrix multiplication without doing all of the multiplications
    num = np.einsum('ij,ji->i', u_matrix, matrix.T)
    u_mag = np.sqrt(np.einsum('ij,ji->i', u_matrix, u_matrix.T))
    mag = np.sqrt(np.einsum('ij,ji->i', matrix, matrix.T))
    denum = u_mag * mag
    
    with np.errstate(invalid='ignore'):
        sim = np.around(num/denum, 6)
        sim[np.isnan(sim)] = 0
    
    return sim

### Calculating similarity score on multiple vectors

One can easily find the similarity on multiple vectors by looping through each one and calling `cos_sim_score`, but I dug deeper and found a way to perform the calculations at the matrix level instead of the vector level. As seen in the cells below, going through each vector is slower than doing a matrix operation.

In [533]:
def cos_sim_scores_ineff(u_vec, matrix):
    results = []
    for v_vec in matrix:
        results.append(cos_sim_score(u_vec, v_vec))
    return np.array(results)

In [786]:
%timeit for _ in range(100): cos_sim_scores_ineff(mutil.matrix[0], mutil.matrix[1:])

5.29 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [787]:
%timeit for _ in range(100): cos_sim_scores(mutil.matrix[0], mutil.matrix[1:])

649 ms ± 37.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [186]:
# Gotcha
# What to do with pearson correlation when the standard deviation is zero for one or both vectors?
# https://stats.stackexchange.com/questions/18333/what-is-the-correlation-if-the-standard-deviation-of-one-variable-is-0

In [183]:
def pear_sim_score(u_vec, v_vec):
    u_avg_rating = average_rating(u_vector)
    v_avg_rating = average_rating(v_vector)
    
    u_ratings_idx = np.where(u_vector != 0)
    v_ratings_idx = np.where(v_vector != 0)
    
    intersecting_idx = np.intersect1d(u_ratings_idx, v_ratings_idx)
    u_ratings = u_vector[intersecting_idx]
    v_ratings = v_vector[intersecting_idx]
    
    u_ratings = u_ratings - u_avg_rating
    v_ratings = v_ratings - v_avg_rating
    
    # In cases where all ratings are zero as a result of subtracting the average, I assume no correlation.
    if not any(v_ratings) or not any(u_ratings):
        return 0
        
    num = np.multiply(u_ratings, v_ratings).sum()
    u_mag = np.sqrt(np.dot(u_ratings, u_ratings))
    v_mag = np.sqrt(np.dot(v_ratings, v_ratings))
    denom = u_mag * v_mag
    return round(num / denom, 6)

In [ ]:
def pear_sim_scores_ineff(u_vec, matrix):
    results = []
    for v_vec in matrix:
        results.append(pear_sim_score((u_vec, v_vec))
    return np.array(results)

In [ ]:
# u_vector = np.array([7, 6, 7, 4, 5, 4])
# v_vector = np.array([0, 3, 3, 1, 1, 0])

## Predicting Ratings

To predict a target user's rating of an item, we rely on the ratings from other users and amplify or attenuate the impact of a rating from those users based on the similarity between them and the target user.

(1.3)
$$ 
\hat{r}_{uj} = µ_u + \frac{\sum_{v∈P_u(j)}Sim(u, j) * r_{vj}}{\sum_{v∈P_u(j)}|Sim(u, j)|}
$$

Where:

$P_u(j)$ is the set of users that are most similar to user **u**. The cardinality of this set could possibly be a max of 10.

$Sim(u, j)$ is a similarity function that returns a score of how similar users u and j are.

$µ_{u}$ is the mean of the ratings for user u.

With equation 1.3 above, we are simply using the ratings of the most similar users to the target user **u**, then using the similarity between user u and those similar users to weigh how important their ratings should be. For example, a user with similarity 0.9 will have a rating with more impact than a user with similarity 0.2.

#### Gotcha
One has to calculate the set $P_u(j)$ for every item. Hence, the set is not a constant value for every user. This is because the most similar users to a target user at a global level, may not have an rating intersection on all items. Hence, calculating the set per item should be kept in mind.

### Happy users and not so happy users.

Some users tend to give high ratings to items regardless of how poor it is, while some tend to give low or average ratings regardless of how good an item is. Having users with tendency to rate low in the similarity set for an item when predicting ratings for a user that rates high can lead to predicting a low rating. To counter this problem, we center the ratings around the mean by subtracting the mean rating of each user from their rating of an item. 

Hence, fixing equation 1.3, we have:

(1.4)
$$ 
\hat{r}_{uj} = µ_u + \frac{\sum_{v∈P_u(j)}Sim(u, j) * s_{vj}}{\sum_{v∈P_u(j)}|Sim(u, j)|} = µ_u + \frac{\sum_{v∈P_u(j)}Sim(u, j) * (r_{vj} - µ_v)}{\sum_{v∈P_u(j)}|Sim(u, j)|}
$$



In [617]:
def get_similarity_data(user_id, product_id, mutil, k=10):
    user_idx = mutil.user_id_to_idx.get(user_id)
    product_idx = mutil.product_id_to_idx.get(product_id)
    above = mutil.matrix[:user_idx]
    above = above[above[:,product_idx] != 0]
    below = mutil.matrix[user_idx+1:]
    below = below[below[:,product_idx] != 0]    
    # candidate_matrix = np.concatenate((above, below))
    candidate_matrix = below
    return candidate_matrix
    user_vector = mutil.matrix[user_idx]
    # similarity_scores = 

In [618]:
def predict_rating(user_id, product_id, sim_func):
    pass

In [619]:
mt = get_similarity_data(103639, 1, mutil)

In [790]:
mutil.user_id_to_idx.get(103639)

AttributeError: 'MatrixUtil' object has no attribute 'user_id_to_idx'

In [ ]:
u_vector = matrix.get_user_vector(u_id)
v_vector = matrix.get_user_vector(v_id)

In [192]:
mutil.get_user_vector(103639)

array([0, 0, 0, 5, 5, 4, 0, 1, 0, 0, 1, 1, 5, 1, 0, 0, 0, 5, 0, 0, 0, 0,
       5, 0, 1, 0, 1, 1, 0, 0, 4, 4, 0, 1, 1, 0, 0, 1, 0, 0, 4, 4, 0, 4,
       4, 0, 4, 0, 4, 5, 4, 0, 0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 5, 1, 0, 4,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 4, 1, 0, 0, 0, 5, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 5, 4, 1, 0, 0, 0, 0, 0, 5, 0, 1, 0, 1, 5,
       5, 0, 0, 4, 0, 4, 1, 0, 0, 0, 0, 1, 4, 4, 0, 1, 0, 0, 0, 5, 1, 1,
       1, 1, 0, 0, 0, 4, 4, 0, 1, 4, 1, 4, 1, 4, 1, 1, 1, 4, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 4, 0, 0, 0, 1, 0, 1, 4, 1, 0, 1, 0, 1, 0, 1, 4,
       0, 5, 0, 0, 1, 4, 5, 0, 5, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       5, 5, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 0, 5,
       4, 0, 5, 0, 1, 0, 1, 4, 5, 4, 4, 0, 0, 0, 1, 4, 1, 0, 0, 0, 4, 0,
       0, 0, 0, 5, 0, 0, 1, 0, 5, 5, 0, 4, 0, 4, 1, 0, 4, 4, 0, 0, 1, 1,
       0, 1, 1, 4, 1, 1, 1, 4, 5, 1, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 5, 5,
       0, 0, 1, 0, 0, 1, 4, 0, 4, 4, 0, 4, 5, 0, 1,

In [184]:
# 103954
# 103603
# 103654
# 103806

# Test vectors
# u_vector = np.array([7, 6, 7, 4, 5, 4])
# v_vector = np.array([0, 3, 3, 1, 1, 0])

# Item-based collaborative filtering

The items similar to a target item are retrieved. Then the user's ratings on those similar items extracted with their weighted average calculated. This calculated average becomes the predicted rating for that item.